In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc

from CS2_bo_functions import bo_iter_w_runs
from CS2_bo_functions import find_train_doc_path
from CS2_bo_functions import set_ep

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Theta = np.linspace(-1,1,3)
Theta2 = np.linspace(-1,1,3)
l = list(permutations(Theta,2))
l = list((combinations_with_replacement(Theta, 2)))
# l = list((combinations(Theta, 2)))
print(l)
print(len(l))

[(-1.0, -1.0), (-1.0, 0.0), (-1.0, 1.0), (0.0, 0.0), (0.0, 1.0), (1.0, 1.0)]
6


In [3]:
p=20
Theta1 =  np.linspace(-2,2,p) #1x10
Theta2 =  np.linspace(-2,2,p) #1x10
theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays 
print(theta_mesh.shape)

(2, 20, 20)


In [4]:
%%time
import itertools
Theta = np.linspace(0,1,10)
df = pd.DataFrame(list(itertools.product(Theta, repeat=2)))
df2 = df.drop_duplicates()
theta_list = df2.to_numpy()

CPU times: user 2.28 ms, sys: 62 µs, total: 2.34 ms
Wall time: 152 ms


In [5]:
# print(theta_list)
print(theta_list.shape)

(100, 2)


In [6]:
theta_list2 = theta_list.reshape((10,-1,2)).T
theta_list2.shape

(2, 10, 10)

In [7]:
theta_list2[0]

array([[0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
       [0.        , 0.11111111, 0.2222222

In [8]:
x = np.array([0,1])
J = (np.repeat(x,8))
J.reshape(-1,8).T

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [35]:
# sse = np.array([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]])
sse = np.zeros((8))
print(sse)
argmin = np.array(np.where(np.isclose(sse, np.amin(sse),atol=np.amin(sse)*1e-6)==True))[0]
print(argmin)
rand_ind = np.random.randint(np.max(argmin))
print(np.max(argmin))
print(rand_ind)

[0. 0. 0. 0. 0. 0. 0. 0.]
[0 1 2 3 4 5 6 7]
7
3


In [32]:
argmin2 = argmin[:,rand_ind]
print(argmin2)

[3]


In [24]:
if len(argmin[0]) > 1:
#         rand_ind = np.random.randint(np.max(argmin[0]))
#         argmin = np.array([[argmin[0,rand_ind]],[argmin[1,rand_ind]]])
    #argmin.T[0].T
    argmax = np.array(argmin2)
print(argmax.T)

[0]


In [26]:
argmax_best = np.zeros(len(argmin[:,0]))

for i in range(len(argmin[0])):
    #Create the corresponding argmax point that maps to theta 1 and theta 2 values
    point = np.array(argmin[:,i])
    print(point)
    
    distance_sq = 1
    max_distance_sq = 5
    

    #Set distance to max distance if it is applicable. At the end of the loop, argmax will be the point with the greatest distance.
    if distance_sq > max_distance_sq:
        max_distance_sq = distance_sq
        argmax_best = point
# print(argmax_best)

[0]
[1]


In [13]:
# l_bounds =  [-2, -2, -10 ,-2]
# u_bounds =  [0, 0 , 0, 2 ]
# l_bounds =  [-2, -2]
# u_bounds =  [2, 2]
# bounds = np.array([[-2,-2],
#                    [ 2, 2]])
d = 2
# d = 2
seed = 9
points = 5
def LHS_Design(num_points, dimensions, seed = None, bounds = None):
    """
    Design LHS Samples
    
    Parameters
    ----------
        num_points: int, number of points in LHS, should be greater than # of dimensions
        dimensions: int, number of parameters to be regressed
        bounds (optional): ndarray, array containing upper and lower bounds of elements in LHS sample. Defaults of 0 and 1
        seed (optional): int, seed of random generation
    Returns
    -------
        LHS: ndarray, Array of LHS sampling points
    """
    sampler = qmc.LatinHypercube(d=dimensions, seed = seed)
    LHS = sampler.random(n=num_points)
    
    if bounds is not None:
        LHS = qmc.scale(LHS, bounds[0], bounds[1])

    return LHS

LHS = LHS_Design(points, d, seed)

In [14]:

LHS = np.array([[1,2,2],
                [3,4,3],
                [5,6,5],
                [7,8,7],
                [9,10,9]])
print(LHS)
# print(LHS.reshape(3,-1))

[[ 1  2  2]
 [ 3  4  3]
 [ 5  6  5]
 [ 7  8  7]
 [ 9 10  9]]


In [15]:
a = np.linspace(0,2,3)
l = list(combinations(a, 2))
l = np.array(l)
print(l)

[[0. 1.]
 [0. 2.]
 [1. 2.]]


In [17]:
# LHS_reshape = LHS.reshape(3,-1)
LHS_reshape = LHS
J = np.meshgrid(LHS_reshape[int(l[0,0])],LHS_reshape[int(l[0,1])])

In [18]:
print(J)

[array([[1, 2, 2],
       [1, 2, 2],
       [1, 2, 2]]), array([[3, 3, 3],
       [4, 4, 4],
       [3, 3, 3]])]


In [ ]:
x, y = J
Z = x + y

In [ ]:
print(Z)

In [ ]:
z = []
for i in range(len(l)):
    x, y = np.meshgrid(LHS_reshape[int(l[i,0])],LHS_reshape[int(l[i,1])])
    z.append(x+y)
z = np.array(z)
print(z.shape)

In [21]:
H = np.array([[1,2,3],[1,2,3]])
print(H.shape)
point = np.array([H[0],H[1]])
print(point)

(2, 3)
[[1 2 3]
 [1 2 3]]
